In [1]:
import tensorflow as tf 
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import FileResponse
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import io
import nest_asyncio
from predications import *
import shutil
import os 

In [2]:
tmkc = tf.keras.models.load_model("model_tumor.h5")
alzhimer = tf.keras.models.load_model("alzhimer_model.h5")

In [3]:
users_db = {}
app=FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_headers=["*"],
    allow_methods={"*"}
)



In [4]:

@app.post("/tumor")
async def analyze_tumor(file:UploadFile=File(...)):
    contents= await file.read()
    os.makedirs("saved_images", exist_ok=True)

    save_path = os.path.join("saved_images", file.filename)

    with open(save_path, "wb") as f:
        f.write(contents)

    

    prediction = tumor(contents,tmkc,file.filename)
    
    file_path = "tumor.pdf"  
    if os.path.exists(file_path):
        return FileResponse(path=file_path, media_type='application/pdf', filename="report.pdf")
    else:
        return {"error": "File not found"}

In [5]:
@app.post("/alz")
async def analyze_tumor(file:UploadFile=File(...)):
    contents= await file.read()
    os.makedirs("saved_images", exist_ok=True)

    save_path = os.path.join("saved_images", file.filename)

    with open(save_path, "wb") as f:
        f.write(contents)

    

    prediction = alz(contents,alzhimer,file.filename)
    
    file_path = "alzhmier.pdf"  
    if os.path.exists(file_path):
        return FileResponse(path=file_path, media_type='application/pdf', filename="report.pdf")
    else:
        return {"error": "File not found"}

In [ ]:
nest_asyncio.apply()
if __name__ =="__main__":
    import uvicorn
    uvicorn.run(app,host="0.0.0.0",port=8000)

INFO:     Started server process [21588]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 934ms/step
Image with grid saved at grid_imt.jpg
Centered image PDF saved as tumor.pdf
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Image with grid saved at grid_imt.jpg
Centered image PDF saved as tumor.pdf
INFO:     103.148.1.82:0 - "POST /tumor HTTP/1.1" 200 OK
INFO:     103.148.1.82:0 - "POST /tumor HTTP/1.1" 200 OK
